In [9]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
model=Sequential()
import cv2

In [6]:
import os ,cv2
ds=[]
categories=['with_mask','without_mask']
c=0
for category in categories:
    path=os.path.join('data',category)
    label=categories.index(category)
    for file in os.listdir(path):
        if c==0:
          img_path=os.path.join(path,file)
          img=cv2.imread(img_path,1)
          img=cv2.resize(img,(224,224))
          ds.append([img,label])
          c=c+1;
        else:
          if c<4:
            c+=1
          else:
            if c==4:
              c=0
              
          
     

In [7]:
sum=0
sum1=0
for i in ds:
  if i[1]==0:
    sum=sum+1
  else :
    sum1=sum1+1  

print("Sum :",sum," Sum1: ",sum1)

Sum : 745  Sum1:  766


In [8]:
import random 
random.shuffle(ds)
X=[]
Y=[]

for i in ds:
    X.append(i[0])
    Y.append(i[1])

In [9]:
import numpy as np
X=np.array(X)
Y=np.array(Y)

In [10]:
X=X/255

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [12]:
X=[]
Y=[]

for i in ds:
    X.append(i[0])
    Y.append(i[1])

In [13]:
from keras.applications.vgg16 import VGG16
vgg=VGG16()

In [14]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [15]:
for layer in model.layers:
  layer.trainable=False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [16]:
model.add(Dense(1,activation='sigmoid'))

In [17]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))


Epoch 1/10


38/38 [==============================] - 144s 4s/step - loss: 0.6216 - accuracy: 0.6755 - val_loss: 0.4810 - val_accuracy: 0.8086
Epoch 2/10
38/38 [==============================] - 153s 4s/step - loss: 0.4482 - accuracy: 0.8336 - val_loss: 0.4035 - val_accuracy: 0.8383
Epoch 3/10
38/38 [==============================] - 152s 4s/step - loss: 0.3786 - accuracy: 0.8667 - val_loss: 0.3260 - val_accuracy: 0.9076
Epoch 4/10
38/38 [==============================] - 187s 5s/step - loss: 0.3260 - accuracy: 0.8907 - val_loss: 0.3042 - val_accuracy: 0.9208
Epoch 5/10
38/38 [==============================] - 167s 4s/step - loss: 0.2974 - accuracy: 0.9007 - val_loss: 0.2709 - val_accuracy: 0.9109
Epoch 6/10
38/38 [==============================] - 168s 4s/step - loss: 0.2735 - accuracy: 0.9106 - val_loss: 0.2580 - val_accuracy: 0.9142
Epoch 7/10
38/38 [==============================] - 160s 4s/step - loss: 0.2627 - accuracy: 0.9098 - val_loss: 0.2350 - val_accuracy: 0.9274
Epoch 8/10

In [ ]:
## 0 for web cam 

In [18]:
model.save("mask_detection_model.h5")

C:\Users\teera\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
cap=cv2.VideoCapture(0)
while True:
  ret,frame=cap.read()
  img=cv2.resize(frame,(224,224))
  
  y_pred=detect_mask(img)
  coods=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
  for x,y,w,h in coods:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),thickness=3)
  if(y_pred<0.5):
        draw_label(frame,"Masked",(10,10),(0,255,0))
  else:
    draw_label(frame,"No mask",(10,10),(255,0,0))
  cv2.imshow("Window",frame)
  if cv2.waitKey(1) & 0xFF == ord('x'):
    break
cv2.destroyAllWindows()

1/1 [==============================] - 0s 267ms/step


In [14]:
def detect_mask(img):
    y_pred=model.predict(img.reshape(1,224,224,3))
    
    return y_pred


In [15]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_X=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]-2
    
    cv2.rectangle(img,pos,(end_X,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
    

In [10]:
from keras.models import load_model

In [11]:
model=load_model("mask_detection_model.h5")

In [12]:
# haar=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
haar = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [13]:
def detect_face(img):
    coods=haar.detectMultiScale(img)
    return coods

In [7]:
cap=cv2.VideoCapture(0)
while True:
  ret,frame=cap.read()
  coods=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
  for x,y,w,h in coods:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),thickness=3)
  cv2.imshow("Window",frame)
  if cv2.waitKey(1) & 0xFF == ord('x'):
    break
cv2.destroyAllWindows()